In [16]:
import tensorflow as tf
import numpy

#a simple dense class
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        
        #create a matrix w, of shape(input_size,ouput_size), initialized with random values.
        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.w = tf.Variable(w_initial_value)
        
        #Create a vector, b, of shape (output_size), initialized with zeros
        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b=tf.Variable(b_initial_value)
        
    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.w) + self.b)
    
    @property
    def weights(self):
        return [self.w, self.b]

class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers
    
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    @property
    def weights(self):
        weights = []
        for layers in self.layers:
            weights += layers.weights
        return weights

model = NaiveSequential([
    NaiveDense(input_size=28*28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images)/batch_size)
        
    def next(self):
        images = self.images[self.index: self.index + self.batch_size]
        labels = self.labels[self.index: self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

def one_training_step(mode1, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

learning_rate = 1e-3

def update_weights(gradients,weights):
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate)
        
def fit(model, images, labels, epochs, batch_size = 128):
    for epoch_counter in range(epochs):
        print(f"Epoch {epoch_counter}")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"loss at batch { batch_counter}: {loss:.2f}")
                
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32")/255
test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype("float32")/255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = numpy.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(F"accuracy: {matches.mean():.2f}")

Epoch 0
loss at batch 0: 4.03
loss at batch 100: 2.21
loss at batch 200: 2.16
loss at batch 300: 2.05
loss at batch 400: 2.19
Epoch 1
loss at batch 0: 1.88
loss at batch 100: 1.84
loss at batch 200: 1.78
loss at batch 300: 1.67
loss at batch 400: 1.80
Epoch 2
loss at batch 0: 1.54
loss at batch 100: 1.53
loss at batch 200: 1.46
loss at batch 300: 1.37
loss at batch 400: 1.48
Epoch 3
loss at batch 0: 1.28
loss at batch 100: 1.29
loss at batch 200: 1.20
loss at batch 300: 1.16
loss at batch 400: 1.25
Epoch 4
loss at batch 0: 1.09
loss at batch 100: 1.11
loss at batch 200: 1.00
loss at batch 300: 1.00
loss at batch 400: 1.09
Epoch 5
loss at batch 0: 0.95
loss at batch 100: 0.98
loss at batch 200: 0.87
loss at batch 300: 0.89
loss at batch 400: 0.98
Epoch 6
loss at batch 0: 0.84
loss at batch 100: 0.88
loss at batch 200: 0.77
loss at batch 300: 0.80
loss at batch 400: 0.89
Epoch 7
loss at batch 0: 0.76
loss at batch 100: 0.79
loss at batch 200: 0.69
loss at batch 300: 0.74
loss at batch 40